# DeepKLM : 신경망 언어모델을 활용한 언어 실험을 위한 라이브러리 (가칭)

## 초기 설정

아래 셀을 실행하여 초기 설정을 수행해 주세요.

In [ ]:
!bash ./scripts/setup.sh

In [ ]:
import torch
import sys

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from scripts.surprisal import bert_token_surprisal, bert_sentence_surprisal, confusion_score, confusion_score_batch
from scripts.visualization import attention_heatmap, visualize_attention_head
from scripts.boxplot_creator import draw_box_plot
from scripts.barplot_creator import draw_bar_plot

from sys import platform
from os import path
from torch import device
from transformers import AdamW, BertConfig, BertModel, BertTokenizer, BertForMaskedLM
from bertviz_lin.pytorch_pretrained_bert import BertForTokenClassification

%matplotlib inline

In [ ]:
if platform == "linux" or platform == "linux2": 
    flist = fm.get_fontconfig_fonts()
    available_fonts = [fm.FontProperties(fname=fname).get_name() for fname in flist]
    if 'NanumGothic' in available_fonts:
        plt.rcParams['font.family'] = 'NanumGothic'
    else:
        print("Font NanumGothic was not found... Try installing a font")
        !apt-get update -qq
        !apt-get install fonts-nanum* -qq
        print("Installed the font!")
        fm._rebuild()
        print("=================IMPORTANT==============================")
        print("If on Colab, RESTART THE RUNTIME to apply the font.")
elif platform == "darwin":
    plt.rcParams['font.family'] = 'AppleGothic' 
elif platform == "win32":
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    print("User platform could not be identified. Korean characters may not be shown correctly when visualizing.")

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## 모델 불러오기

### 영어

기본으로 버트(large, uncased)를 불러옵니다.

In [ ]:
mask_model_eng = BertForMaskedLM.from_pretrained('bert-large-uncased', output_attentions=True)
classification_model_eng = BertForTokenClassification.from_pretrained('bert-large-uncased', num_labels=2)
tokenizer_eng = BertTokenizer.from_pretrained('bert-large-uncased')

### 한국어
기본으로 KR-BERT를 불러옵니다.

In [ ]:
modelpath= "./KR-BERT/krbert_pytorch/pretrained/pytorch_model_char16424_ranked.bin"
config = BertConfig.from_json_file("./KR-BERT/krbert_pytorch/pretrained/bert_config_char16424.json")
config.output_attentions = True
tokenizer_kr = BertTokenizer.from_pretrained('./KR-BERT/krbert_pytorch/pretrained/vocab_snu_char16424.txt', do_lower_case=False)
mask_model_kr =BertForMaskedLM.from_pretrained(modelpath,config=config)

ETRI의 KorBERT가 있을 경우, 불러옵니다.

In [ ]:
if path.exists("KorBERT"):
  sys.path.insert(1, "./KorBERT/001_bert_morp_pytorch/src_tokenizer")
  import tokenization_morp
  """IN CASE OF ImportError:
  1. Go to the src_tokenizer.py in KorBERT
  2. Go to line 32 (from .file_utils import cached_path)
  3. Change the line to the following
    from pytorch_pretrained_bert.file_utils import cached_path
  4. Enjoy :)
  """

  korbert_path = "./KorBERT/001_bert_morp_pytorch/"
  modelpath= korbert_path + "pytorch_model.bin"
  config = BertConfig.from_json_file(korbert_path + "bert_config.json")
  mask_model_etri=BertForMaskedLM.from_pretrained(modelpath,config=config)
  tokenizer_etri = tokenization_morp.BertTokenizer.from_pretrained(korbert_path + "vocab.korean_morp.list")
else:
  print("KorBERT not found. Skipping...")

# (1차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [ ]:
text = """
철수가 영희[MASK] 좋아한다."""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["을", "를"], mask_model_kr, tokenizer_kr, device)

# (2차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [ ]:
text = """
철수가 영희[MASK] 좋아한다.
철수는 영희[MASK] 좋아한다."""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["을", "를"], mask_model_kr, tokenizer_kr, device)

# (3차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [ ]:
text = """
철수가 영희[MASK] 좋아한다.
철수가 영희[MASK] 싫어한다.
철수는 영희[MASK] 좋아한다.
철수가 영희[MASK] 싫어한다."""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["을", "를"], mask_model_kr, tokenizer_kr, device)

## 주의할 점

- 컴퓨터의 입장에서 최소대립쌍이 맞는지 확인하여야 합니다.
    - 가령, 철수는 자신을/자기를 사랑한다.에서, 
    - 자신, 자기만 바뀐 것이 아니라
    - 을/를 토큰 또한 바뀌었기 떄문에
    - 최소대립쌍이 아닙니다.
- 버트에 "등록된" 단어인지 확인하여야 합니다.
    - 효율성을 위해 버트는 '바이트 페어 인코딩'이라는 것을 수행합니다.
    - 따라서, 단어가 (형태소와는 상관 없는) 단어보다 작은 단위로 나누어서 등록되어 있을 수 있습니다.
    - 키워드가 \[UNK\]로 인식되지 않았는지 확인하셔야 합니다.

## Confusion Score
from Lin et al. (2019)

In [ ]:
confusion_score("The scholar that published the paper has ever resigned the position	0	7	4", classification_model_eng, tokenizer_eng)

## 시각화
* 히트맵 시각화는 attention_heatmap()
    * 입력값은 리스트([ ]) 안에 들어가야 합니다.
    * vis_opt값은 0, 1, 2 중 하나로, 히트맵의 형태를 결정합니다.
* BertViz는 visualize_attention_head()

In [ ]:
attention_heatmap(["학생들이 통사론 과제를 하지 않았다"],
                  mask_model_kr, tokenizer_kr, device, vis_opt=2)

In [ ]:
visualize_attention_head(mask_model_kr, tokenizer_kr, "학생들은 통사론 과제를 제출했다")

## 플롯 그리기
### 박스플롯
* 제공된 template.xlsx 파일을 결과로 채워 주세요.
    * 2by2 실험의 경우 factor1까지 채우시면 됩니다.
        * factor2는 비워 두세요
    * 2by2by2 실험의 경우 factor2까지 채우셔야 합니다.
* boxplot_config.txt 파일을 수정해 주세요
    * filepath = _파일 이름_
    * factor1 = _첫 번째 요인 이름_
    * factor1_vals = _첫 번째 요인의 변수_
    * factor2 = _두 번째 요인 이름_
        * 2by2 실험의 경우 비워 두세요
    * factor2_vals = _두 번째 요인의 변수_
        * 2by2 실험의 경우 비워 두세요
    * variables_value = _변수 이름_
    * mask_vals = _마스크에 들어갈 키워드_
    * notch = _노치를 추가하기 위해서는 True로 설정_
    * title = _플롯 타이틀_
    * size = _2by2 실험은 22; 2by2by2 실험은 222_
* 결과 플롯은 같은 디렉토리에 저장됩니다.
* 기본적으로, 같이 제공된 부정극어 사례의 결과를 보여줍니다.

In [ ]:
draw_box_plot()

### 바플롯

* barplot_sample.xlsx과 같이 데이터를 정리하십시오.

In [ ]:
draw_bar_plot("barplot_sample.xlsx")

# 참고문헌

- Lin, Y., Tan, Y. C., & Frank, R. (2019). Open Sesame: Getting Inside BERT's Linguistic Knowledge. arXiv preprint arXiv:1906.01698.